In [9]:
import json
import glob
import os
import pandas as pd
from pathlib import Path
import pickle
import shutil
import subprocess
from tempfile import TemporaryDirectory
import time
from tqdm import tqdm

In [2]:
# Path to the oss detect backdoor binary
oss_detect_backdoor_path = os.path.abspath("../scanners/OSSGadget/src/oss-detect-backdoor/bin/Debug/net6.0/oss-detect-backdoor")

In [61]:
# Scanning packages, save the results to disk, and record running times
def scanning_packages(package_releases_path, results_dir):
    running_times = {}
    package_dirs = [f.path for f in os.scandir(package_releases_path) if f.is_dir()]
    for package_dir in tqdm(package_dirs):
        package_name = f"{package_dir.split('/')[6]}"
        print(package_name)
        # As there a big packages containing non python files, we select only python files to analyze
        with TemporaryDirectory() as temp_dir:
            for dirpath,_,filenames in os.walk(package_dir):
                for f in filenames:
                    file_path = os.path.abspath(os.path.join(dirpath, f))
                    if Path(file_path).suffix == '.py':
                        shutil.copy2(file_path, temp_dir)
            start_time = time.time()
            subprocess.run([oss_detect_backdoor_path, temp_dir, "-f", "sarifv2", "-o", f"{os.path.join(results_dir, package_name)}.sarif"])
            running_times[package_name] = time.time() - start_time
    return running_times

In [79]:
def collect_alerts(results_dir):
    alerts = []
    for root, dirs, files in os.walk(results_dir):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.endswith(".sarif"):
                package_name = file_path.split("/")[-1].replace(".sarif", "")
                print(package_name)
                alert_data = json.load(open(file_path))
                alert_data["package_name"] = package_name
                alerts.append(alert_data)
    return alerts

In [94]:
def process_alerts(alerts):
    processed_alerts = []
    for data in alerts:
        results = data["runs"][0]["results"]
        package_name = data["package_name"]
        if len(results) > 0:
            for result in results:
                alert = result["message"]["text"]
                for loc in result["locations"]:
                    target = loc['physicalLocation']["address"]["fullyQualifiedName"]
                    if target.endswith(".py"):
                        processed_alerts.append([package_name, target, alert])
        else:
            print(f"{package_name} does not have alerts")
    return processed_alerts

In [12]:
def get_tp_fp(num_alerts_list, threshold):
    count = 0
    for i in num_alerts_list:
        if i > threshold:
            count = count + 1
    tp = count
    fp = len(num_alerts_list) - tp
    return (tp, fp)

## Malicious packages

In [74]:
# Paths to malicious packages and the scanning results
malicious_packages_path = os.path.abspath("../dataset/malicious-packages/")
malicious_results_dir = os.path.abspath("../results/oss-detect-backdoor/malicious-packages/")

In [63]:
# Scanning packages, and record runtimes
malicious_packages_running_times = scanning_packages(malicious_packages_path, malicious_results_dir)

  0%|                                                                                                                                                                                                                | 0/168 [00:00<?, ?it/s]

10Cent10

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  1%|█▏                                                                                                                                                                                                      | 1/168 [00:01<05:17,  1.90s/it]

10Cent11

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  1%|██▍                                                                                                                                                                                                     | 2/168 [00:03<04:59,  1.81s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 166/168 [06:30<00:04,  2.17s/it]

yandex-yt

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 167/168 [06:32<00:02,  2.16s/it]

yiffparty

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [06:34<00:00,  2.35s/it]


In [16]:
# Runtime statistics
malicious_packages_running_times_df = pd.DataFrame(list(malicious_packages_running_times.items()), columns=['package', 'running_time'])
malicious_packages_running_times_df["running_time"].describe()

count    168.000000
mean       1.882648
std        0.699136
min        1.262043
25%        1.471162
50%        1.581486
75%        2.046860
max        4.899373
Name: running_time, dtype: float64

In [44]:
# Saving runtime results
malicious_packages_running_times_df.to_csv("../results/running_times/oss-detect-backdoor/malicious-packages.csv")

count    168.000000
mean       1.839182
std        0.637643
min        1.109901
25%        1.366614
50%        1.757555
75%        1.980360
max        5.564219
Name: running_time, dtype: float64

In [89]:
# Analyzing maicious packaes results
malicious_results_path = os.path.abspath("../results/oss-detect-backdoor/malicious-packages/")
malicious_results = collect_alerts(malicious_results_path)

10Cent10
10Cent11
AadhaarCrypt
acqusition
aiohttp-socks5
aiohttp_proxies
aiohttp_socks4
aml-ds-pipeline-contrib
apidev-coop
apple-py-music
arduino
are
aryi
asdeefdzfzs
aws-login0tool
azure-cli-ml-preview
azure-cli-ml-private-preview
azureml-contrib-jupyterrun
azureml-contrib-optimization
azureml-contrib-reports
beautfulsoup
botaa3
bs4tools
bzip
collored
colourama
covd
crypt
ctx
cupy-cuda112
dajngo
dark-magic
diango
discordcmd
DiscordSafety
distrib
distutil
djago
djanga
django-server
dpp_client
dpp_client1234
easyfuncsys
easyinstall
ecopower
fakessh
flask-requests-complex
fleshh
floogle
gearbest_parser
genesisbot
hipid
hkg-sol-utils
huehuehuehue
humanqueen
humanqueenn
important-package
importantpackage
infoglmi
ipahttp
ipboards
jajajejejiji
jeilyfish
junkeldat
karapace
learninglib
libcurl
libffm
libhtml5
libpeshka
libpeshnx
librat
maratlib
maratlib1
mateplotlib
matplatlib-plus
mllearnlib
mplatlib
multilingual_t5
mumpy
mybiubiubiu
my_project100
my_project10101
my_project101011
nmap-pytho

In [95]:
# Processing malicious alerts
malicous_packages_scanning_results = process_alerts(malicious_results)

10Cent10 does not have alerts
10Cent11 does not have alerts
are does not have alerts
aws-login0tool does not have alerts
discordcmd does not have alerts
ecopower does not have alerts
hkg-sol-utils does not have alerts
importantpackage does not have alerts
jajajejejiji does not have alerts
learninglib does not have alerts
libffm does not have alerts
libpeshka does not have alerts
libpeshnx does not have alerts
mllearnlib does not have alerts
my_project10101 does not have alerts
owlmoon does not have alerts
pip_security does not have alerts
pygrata does not have alerts
pymafka does not have alerts
pyscrapy does not have alerts
pytagora does not have alerts
pytagora2 does not have alerts
rstseuqe does not have alerts
setuptool does not have alerts
yandex-yt does not have alerts


### Triggered rules in all Python files in malicious packages

In [ ]:
# Loading the existing result in case we do not want to rescan the packages
with open(os.path.abspath(os.path.join("..", "results", "oss-detect-backdoor", "malicious-packages.pkl")), 'rb') as fp:
    malicous_packages_scanning_results = pickle.load(fp)

In [96]:
malicious_results_df = pd.DataFrame(malicous_packages_scanning_results, columns=["package", "target", "rule"])

In [98]:
# Number of rules per package
malicious_packages_rules_groupby = malicious_results_df.groupby('package')['rule']
print(f"Total number of rules: {malicious_packages_rules_groupby.count().sum()}")
malicious_packages_rules_groupby.count().describe()

Total number of rules: 7869


count    143.000000
mean      55.027972
std      146.182885
min        1.000000
25%        3.000000
50%        7.000000
75%       19.000000
max      858.000000
Name: rule, dtype: float64

### Triggered rules in all setup.py files in malicious packages

In [99]:
malicious_packages_rules_setup_df = malicious_results_df[malicious_results_df['target'].str.contains('setup.py')]

In [100]:
# Number of rules per package
malicious_packages_rules_setup_groupby = malicious_packages_rules_setup_df.groupby('package')['rule']
print(f"Total number of rules: {malicious_packages_rules_setup_groupby.count().sum()}")
malicious_packages_rules_setup_groupby.count().describe()

Total number of rules: 622


count     85.000000
mean       7.317647
std       18.000801
min        1.000000
25%        2.000000
50%        4.000000
75%        7.000000
max      161.000000
Name: rule, dtype: float64

In [ ]:
# Saving the existing result
with open(os.path.abspath("../results/malicious_packages_scanning_results_oss_detect_backdoor.pkl"), 'wb') as fp:
    pickle.dump(malicious_results, fp, protocol=pickle.HIGHEST_PROTOCOL)

## Popular packages

In [102]:
# Paths to popular packages and results dir
popular_packages_path = os.path.abspath("../dataset/popular-packages/")
popular_results_dir = os.path.abspath("../results/oss-detect-backdoor/popular-packages/")

In [103]:
# Scanning packages, and record runtimes
popular_packages_running_times = scanning_packages(popular_packages_path, popular_results_dir)

  0%|                                                                                                                                                                                                               | 0/1430 [00:00<?, ?it/s]

absl-py

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  0%|▏                                                                                                                                                                                                    | 1/1430 [00:03<1:11:27,  3.00s/it]

accelerate

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  0%|▎                                                                                                                                                                                                    | 2/1430 [00:06<1:19:55,  3.36s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 1428/1430 [1:46:39<00:06,  3.13s/it]

zstandard

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 1429/1430 [1:46:42<00:02,  2.93s/it]

zthreading

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1430/1430 [1:46:44<00:00,  4.48s/it]


In [30]:
# Measuring runtimes of scanning popular packages
popular_packages_running_times_df = pd.DataFrame(list(popular_packages_running_times.items()), columns=['package', 'running_time'])
popular_packages_running_times_df["running_time"].describe()

In [31]:
popular_packages_running_times_df.to_csv("../results/running_times/oss-detect-backdoor/popular-packages.csv")

In [51]:
# Analyzing the results
popular_results_path = os.path.abspath("../results/oss-detect-backdoor/popular-packages/")
popular_results = collect_alerts(popular_results_path)

alabaster does not have alerts
attrdict does not have alerts
aws-cdk-lib does not have alerts
aws-cdk.aws-ec2 does not have alerts
aws-cdk.aws-events does not have alerts
aws-cdk.aws-iam does not have alerts
aws-cdk.aws-kms does not have alerts
aws-cdk.aws-lambda does not have alerts
aws-cdk.aws-logs does not have alerts
aws-cdk.aws-s3 does not have alerts
aws-cdk.cdk does not have alerts
aws-cdk.core does not have alerts
azure-cli-command-modules-nspkg does not have alerts
azure-cli-nspkg does not have alerts
azure-keyvault does not have alerts
azure-mgmt-datalake-nspkg does not have alerts
azure-mgmt-nspkg does not have alerts
azure-nspkg does not have alerts
backports-tempfile does not have alerts
boto3-type-annotations does not have alerts
bs4 does not have alerts
bumpversion does not have alerts
click-didyoumean does not have alerts
click-help-colors does not have alerts
cligj does not have alerts
colornamer does not have alerts
constructs does not have alerts
crashtest does not h

In [ ]:
# Processing malicious alerts
popular_packages_scanning_results = process_alerts(popular_results)

### Triggered rules in all Python files in popular packages

In [ ]:
# Loading the existing result in case we do not want to rescan the packages
with open(os.path.abspath(os.path.join("..", "results", "oss-detect-backdoor", "popular-packages.pkl")), 'rb') as fp:
    popular_packages_scanning_results = pickle.load(fp)

In [7]:
# Transform the results to DataFrame for analysis
popular_results_df = pd.DataFrame(popular_packages_scanning_results, columns=["package", "target", "rule"])

In [8]:
# Number of rules per package
popular_packages_rules_groupby = popular_results_df.groupby('package')['rule']
print(f"Total number of rules: {popular_packages_rules_groupby.count().sum()}")
popular_packages_rules_groupby.count().describe()

Total number of rules: 696755


count      1381.000000
mean        504.529327
std        3806.342235
min           1.000000
25%          19.000000
50%          85.000000
75%         293.000000
max      124267.000000
Name: rule, dtype: float64

In [42]:
# Ratio of true positives to false postives 
thresholds = [1, 2, 3, 4, 5]
scores = popular_packages_rules_groupby.count().to_list()
thesholds_tpr_fpr_ratio = []
for t in thresholds:
    tpr, fpr = get_tpr_fpr(scores, t)
    print(t, round(tpr/fpr, 2))

1 37.36
5 8.09
10 4.73
15 3.5
20 2.84
25 2.44
30 2.12


### Triggered rules in all setup.py files in popular packages

In [9]:
# Select only setup.py files of the packages
popular_packages_rules_setup_df = popular_results_df[popular_results_df['target'].str.contains('setup.py')]

In [10]:
# Number of rules per package
popular_packages_rules_setup_groupby = popular_packages_rules_setup_df.groupby('package')['rule']
print(f"Total number of rules: {popular_packages_rules_setup_groupby.count().sum()}")
popular_packages_rules_setup_groupby.count().describe()

Total number of rules: 2584


count    596.000000
mean       4.335570
std        8.066206
min        1.000000
25%        1.000000
50%        2.000000
75%        4.000000
max       77.000000
Name: rule, dtype: float64

In [46]:
# Ratio of true positives to false postives 
thresholds = [1, 2, 3, 4, 5]
scores = popular_packages_rules_setup_groupby.count().to_list()
thesholds_tpr_fpr_ratio = []
for t in thresholds:
    tp, fp = get_tp_fp(scores, t)
    print(t, tp, fp)

1 331 265
2 225 371
3 156 440
4 130 466
5 116 480


In [20]:
# Saving the existing result
with open(os.path.abspath("../results/oss-detect-backdoor/popular-packages.pkl"), 'wb') as fp:
    pickle.dump(popular_results, fp, protocol=pickle.HIGHEST_PROTOCOL)

## Random packages

In [ ]:
# Loading the existing result in case we do not want to rescan the packages
with open(os.path.abspath(os.path.join("..", "results", "oss-detect-backdoor", "random-packages.pkl")), 'rb') as fp:
    popular_packages_scanning_results = pickle.load(fp)

In [106]:
# Paths to random packages and results dir
random_packages_path = os.path.abspath("../dataset/random-packages/")
random_results_dir = os.path.abspath("../results/oss-detect-backdoor/random-packages//")

In [ ]:
# Scanning packages, and record runtimes
random_packages_running_times = scanning_packages(random_packages_path, random_results_dir)

  0%|                                                                                                                                           | 0/986 [00:00<?, ?it/s]

650-auto-comp-jaewon

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  0%|▏                                                                                                                                  | 1/986 [00:02<34:59,  2.13s/it]

a38-decimal

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


  0%|▎                                                                                                                                  | 2/986 [00:03<32:05,  1.96s/it]

...

sorting-asmaa

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 823/986 [26:08<04:57,  1.83s/it]

spectralpy

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


 84%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▊                     | 824/986 [26:10<04:52,  1.81s/it]

sphinx-multiproject

   ____   _____ _____    _____           _            _
  / __ \ / ____/ ____|  / ____|         | |          | |
 | |  | | (___| (___   | |  __  __ _  __| | __ _  ___| |_
 | |  | |\___ \\___ \  | | |_ |/ _` |/ _` |/ _` |/ _ \ __|
 | |__| |____) |___) | | |__| | (_| | (_| | (_| |  __/ |_
  \____/|_____/_____/   \_____|\__,_|\__,_|\__, |\___|\__|
                                            __/ |
                                           |___/          
OSS Gadget - oss-detect-backdoor 0.1.338+9f1c90696a - github.com/Microsoft/OSSGadget


In [37]:
random_packages_running_times_df = pd.DataFrame(list(random_packages_running_times.items()), columns=['package', 'running_time'])
random_packages_running_times_df["running_time"].describe()

In [38]:
random_packages_running_times_df.to_csv("../results/running_times/oss-detect-backdoor/random-packages.csv")

count    986.000000
mean       2.295999
std        3.608086
min        1.212894
25%        1.540764
50%        1.863567
75%        2.273316
max       69.932577
Name: running_time, dtype: float64

In [13]:
random_results_path = os.path.abspath("../results/oss-detect-backdoor/random/")
random_results = analyze_results(random_results_path)

ahtest-distributions does not have alerts
aicovidvn-2021 does not have alerts
alou-example1 does not have alerts
analyticslab does not have alerts
anyks-sc does not have alerts
arm-backend-cpplib does not have alerts
ashser-athletelist does not have alerts
async-http2 does not have alerts
asyncio-atexit does not have alerts
augmented-startups-one does not have alerts
automagica2 does not have alerts
aws-cdk-aws-codepipeline does not have alerts
aws-solutions-constructs-aws-sns-lambda does not have alerts
basicsorts does not have alerts
binom-probability does not have alerts
brazil-types does not have alerts
camcalib-jpvolt does not have alerts
cdk-metaflow does not have alerts
ceremony does not have alerts
cf does not have alerts
ch-json-logger does not have alerts
chem-tom-first-package-here-now does not have alerts
chengfa does not have alerts
codev-dashboard does not have alerts
confdir does not have alerts
converterjbundy does not have alerts
convnetional-ticket does not have alert

### Triggered rules in all Python files in random packages

In [14]:
# Transform the results to DataFrame for analyssi
random_results_df = pd.DataFrame(random_results, columns=["package", "target", "rule"])

In [15]:
# Number of rules per package
random_packages_rules_groupby = random_results_df.groupby('package')['rule']
print(f"Total number of rules: {random_packages_rules_groupby.count().sum()}")
random_packages_rules_groupby.count().describe()

Total number of rules: 55093


count     761.000000
mean       72.395532
std       386.339730
min         1.000000
25%         5.000000
50%        13.000000
75%        40.000000
max      8100.000000
Name: rule, dtype: float64

In [39]:
# Ratio of true positives to false postives 
thresholds = [1, 5, 10, 15, 20, 25, 30]
scores = random_packages_rules_groupby.count().to_list()
thesholds_tpr_fpr_ratio = []
for t in thresholds:
    tpr, fpr = get_tpr_fpr(scores, t)
    print(t, round(tpr/fpr, 2))

1 10.71
5 2.37
10 1.25
15 0.82
20 0.64
25 0.54
30 0.43


### Triggered rules in all setup.py files in random packages

In [17]:
# Select only setup.py files
random_packages_rules_setup_df = random_results_df[random_results_df['target'].str.contains('setup.py')]

In [18]:
# Number of rules per package
random_packages_rules_setup_groupby = random_packages_rules_setup_df.groupby('package')['rule']
print(f"Total number of rules: {random_packages_rules_setup_groupby.count().sum()}")
random_packages_rules_setup_groupby.count().describe()

Total number of rules: 556


count    184.000000
mean       3.021739
std        4.593838
min        1.000000
25%        1.000000
50%        1.000000
75%        3.000000
max       40.000000
Name: rule, dtype: float64

In [47]:
# Ratio of true positives to false postives 
thresholds = [1, 2, 3, 4, 5]
scores = random_packages_rules_setup_groupby.count().to_list()
thesholds_tpr_fpr_ratio = []
for t in thresholds:
    tp, fp = get_tp_fp(scores, t)
    print(t, tp, fp)

1 88 96
2 58 126
3 35 149
4 25 159
5 19 165


In [19]:
# Saving the existing result
with open(os.path.abspath("../results/oss-detect-backdoor/random-packages.pkl"), 'wb') as fp:
    pickle.dump(random_results, fp, protocol=pickle.HIGHEST_PROTOCOL)